This notebook is part of the `deepcell-tf` documentation: https://deepcell.readthedocs.io/.

# Nuclear segmentation and tracking

In [1]:
import os

import numpy as np

import imageio
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.keras import backend as K

## Prepare nuclear data

Use `imageio` to load each frame of a gif to form a test dataset.

In [2]:
from deepcell.datasets.tracked import hela_s3

(X_train, y_train), (X_test, y_test) = hela_s3.load_tracked_data(seed=0)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
6370648064/6370641920 [==============================] - 225s 0us/step
X_train shape: (144, 40, 216, 256, 1)
X_test shape: (36, 40, 216, 256, 1)


In [3]:
print(X_train.shape)

(144, 40, 216, 256, 1)


In [4]:
x = X_train[56]  # chosen batch with divisions

In [5]:
def plot(im):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(im)
    plt.axis('off')
    plt.title('Raw Image Data')

    fig.canvas.draw()  # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)

    return image

imageio.mimsave('raw.gif', [plot(x[i, ..., 0]) for i in range(x.shape[0])])

### View .GIF of raw cells

![Raw Gif](./raw.gif)

## Nuclear Segmentation

### Initialize nuclear model

The application will download pretrained weights for nuclear segmentation. For more information about application objects, please see our [documentation](https://deepcell.readthedocs.io/en/master/API/deepcell.applications.html).

In [6]:
from deepcell.applications import NuclearSegmentation

app = NuclearSegmentation()

101310464/101306776 [==============================] - 30s 0us/step


## Use the application to generate labeled images

Typically, neural networks perform best on test data that is similar to the training data. In the realm of biological imaging, the most common difference between datasets is the resolution of the data measured in microns per pixel. The training resolution of the model can be identified using `app.model_mpp`.

In [7]:
print('Training Resolution:', app.model_mpp, 'microns per pixel')

Training Resolution: 0.65 microns per pixel


The resolution of the input data can be specified in `app.predict` using the `image_mpp` option. The `Application` will rescale the input data to match the training resolution and then rescale to the original size before returning the labeled image.

In [8]:
y_pred = app.predict(x, image_mpp=0.65)

print(y_pred.shape)

(40, 216, 256, 1)


### Save labeled images as a gif to visualize

In [9]:
def plot(im1, im2, vmin, vmax):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(im1)
    ax[0].axis('off')
    ax[0].set_title('Raw')
    ax[1].imshow(im2, cmap='jet', vmin=vmin, vmax=vmax)
    ax[1].set_title('Segmented')
    ax[1].axis('off')

    fig.canvas.draw()  # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)

    return image

imageio.mimsave(
    './labeled.gif',
    [plot(x[i,...,0], y_pred[i,...,0], y_pred.min(), y_pred.max())
     for i in range(y_pred.shape[0])]
)

### View .GIF of segmented cells

The `NuclearSegmentation` application was able to create a label mask for every cell in every frame!

![Segmented GIF](./labeled.gif)

## Cell Tracking

The `NuclearSegmentation` worked well, but the cell labels of the same cell are not preserved across frames. To resolve this problem, we can use the `CellTracker`! This object will use another `CellTrackingModel` to compare all cells and determine which cells are the same across frames, as well as if a cell split into daughter cells.

### Normalize raw data to prepare for tracking

The `CellTracker` expects input image data to be zero-mean and unit-variance.

In [10]:
from deepcell_toolbox.processing import normalize

x = x.astype('float32')
x_norm = np.empty(x.shape)

for frame in range(x.shape[0]):
    normalized = normalize(x[frame, ..., 0])
    x_norm[frame] = np.expand_dims(normalized, axis=-1)

### Initalize tracking model

Create an instance of `deepcell.applications.CellTrackingModel` and pass the model to the `CellTracker`.

In [11]:
from deepcell.applications import CellTrackingModel
from deepcell_tracking import CellTracker

tracking_model = CellTrackingModel()

cell_tracker = CellTracker(
    x_norm, y_pred, tracking_model,
    birth=0.99, death=0.99, division=0.9)

2269184/2267544 [==============================] - 0s 0us/step


### Track the cells

In [12]:
cell_tracker.track_cells()

/usr/local/lib/python3.6/dist-packages/deepcell_tracking/tracking.py:721: FutureWarning: The coordinates keyword argument to skimage.measure.regionprops is deprecated. All features are now computed in rc (row-column) coordinates. Please remove `coordinates="rc"` from all calls to regionprops before updating scikit-image.
  props = regionprops(np.squeeze(roi), coordinates='rc')[0]
/usr/local/lib/python3.6/dist-packages/deepcell_tracking/tracking.py:685: FutureWarning: The coordinates keyword argument to skimage.measure.regionprops is deprecated. All features are now computed in rc (row-column) coordinates. Please remove `coordinates="rc"` from all calls to regionprops before updating scikit-image.
  props = regionprops(np.squeeze(roi), coordinates='rc')


### Visualize tracking results

In [13]:
# Convert tracking results to a dictionary
data = cell_tracker._track_review_dict()

X = data['X']  # raw X data
y = data['y_tracked']  # tracked y data

In [14]:
def plot(im1,im2,vmin,vmax):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(im1)
    ax[0].axis('off')
    ax[0].set_title('Raw')
    ax[1].imshow(im2, cmap='jet', vmin=vmin, vmax=vmax)
    ax[1].set_title('Tracked')
    ax[1].axis('off')

    fig.canvas.draw()  # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)

    return image

imageio.mimsave('tracks.gif', [plot(X[i,...,0], y[i,...,0], y.min(), y.max())
                               for i in range(y_pred.shape[0])])

### View .GIF of tracked cells

Now that we've finished using `CellTracker.track_cells`, not only do the annotations preserve label across frames, but the lineage information has been saved in `CellTracker.tracks`.

![Tracked Cells GIF](./tracks.gif)